In [128]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs, make_circles
from sklearn.metrics import accuracy_score, log_loss
from tqdm import tqdm
import csv

# Fonctions

In [129]:
def initialisation(dimensions):
    
    parametres = {}
    C = len(dimensions)

    np.random.seed(1)

    for c in range(1, C):
        parametres['W' + str(c)] = np.random.randn(dimensions[c], dimensions[c - 1])
        parametres['b' + str(c)] = np.random.randn(dimensions[c], 1)


    return parametres

In [130]:
def forward_propagation(X, parametres):
  
  activations = {'A0': X}
  
  C = len(parametres) // 2

  for c in range(1, C + 1):

    Z = parametres['W' + str(c)].dot(activations['A' + str(c - 1)]) + parametres['b' + str(c)]
    activations['A' + str(c)] = 1 / (1 + np.exp(-Z))

  return activations

In [131]:
def back_propagation(y, parametres, activations):

  m = y.shape[1]
  C = len(parametres) // 2

  dZ = activations['A' + str(C)] - y
  gradients = {}

  for c in reversed(range(1, C + 1)):
    gradients['dW' + str(c)] = 1/m * np.dot(dZ, activations['A' + str(c - 1)].T)
    gradients['db' + str(c)] = 1/m * np.sum(dZ, axis=1, keepdims=True)
    if c > 1:
      dZ = np.dot(parametres['W' + str(c)].T, dZ) * activations['A' + str(c - 1)] * (1 - activations['A' + str(c - 1)])

  return gradients

In [132]:
def update(gradients, parametres, learning_rate):

    C = len(parametres) // 2

    for c in range(1, C + 1):
        parametres['W' + str(c)] = parametres['W' + str(c)] - learning_rate * gradients['dW' + str(c)]
        parametres['b' + str(c)] = parametres['b' + str(c)] - learning_rate * gradients['db' + str(c)]

    return parametres

In [143]:
def predict(X, parametres):
  activations = forward_propagation(X, parametres)
  C = len(parametres) // 2
  Af = activations['A' + str(C)]
  #>= 0.5
  return Af 

In [151]:
def deep_neural_network(X, y,  n_iter, hidden_layers , learning_rate = 0.001):

    # initialisation parametres
    dimensions = list(hidden_layers)
    dimensions.insert(0, X.shape[0])
    dimensions.append(y.shape[0])
    np.random.seed(1)
    parametres = initialisation(dimensions)

    print(len(parametres))
    
    # tableau numpy contenant les futures accuracy et log_loss
    training_history = np.zeros((int(n_iter), 2))

    C = len(parametres) // 2

    # gradient descent
    for i in tqdm(range(n_iter)):

        activations = forward_propagation(X, parametres)
        gradients = back_propagation(y, parametres, activations)
        parametres = update(gradients, parametres, learning_rate)
        Af = activations['A' + str(C)]



        """ # calcul du log_loss et de l'accuracy
        training_history[i, 0] = (log_loss(y.flatten(), Af.flatten()))
        y_pred = predict(X, parametres)
        training_history[i, 1] = (accuracy_score(y.flatten(), y_pred.flatten()))

    # Plot courbe d'apprentissage
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(training_history[:, 0], label='train loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(training_history[:, 1], label='train acc')
    plt.legend()
    plt.show() """

    print(parametres)
    return training_history

# Dataset

In [152]:


with open('./master_camp_db_femmes/nombre_de_repas/perte.csv','r') as f:

    data = list(csv.reader(f))
    print(data[0])
    
    data = data[1:]
    for el in data:
        """ a = int(int(el[-2])/int(el[-1]))
        b = int((int(el[-2])+150)/int(el[-1]))
        c = [a,b]
         """
        el = list(map(int,el))
        

data = np.array(data,dtype=float)

t = len(data[0])-1

X = data[:,:t]
y = data[:,-1]

X = X.T
y = y.reshape((1, y.shape[0]))

print('dimensions de X:', X.shape)
print('dimensions de y:', y.shape)



#plt.scatter(X[:,0],X[:,1] ,c=y, cmap='summer')
#
# 
# plt.show()

['age', 'poids', 'taille', 'sexe', 'type_regime', 'kcal_cible', 'nb_repas_cible']
dimensions de X: (6, 1000)
dimensions de y: (1, 1000)


In [153]:
deep_neural_network(X, y, hidden_layers = (3,2,1), learning_rate = 0.1, n_iter = 1000)


8


  0%|          | 0/1000 [00:00<?, ?it/s]/tmp/ipykernel_82144/1270483671.py:10: RuntimeWarning: overflow encountered in exp
  activations['A' + str(c)] = 1 / (1 + np.exp(-Z))
100%|██████████| 1000/1000 [00:00<00:00, 2938.04it/s]

{'W1': array([[ 1.62434536, -0.61175641, -0.52817175, -1.07296862,  0.86540763,
        -2.3015387 ],
       [ 1.74481176, -0.7612069 ,  0.3190391 , -0.24937038,  1.46210794,
        -2.06014071],
       [-0.3224172 , -0.38405435,  1.13376944, -1.09989127, -0.17242821,
        -0.87785842]]), 'b1': array([[ 0.04221375],
       [ 0.58281521],
       [-1.10061918]]), 'W2': array([[ 1.14472371,  0.90159072,  0.50249434],
       [ 0.90085595, -0.68372786, -0.12289023]]), 'b2': array([[1.73720872],
       [0.96880719]]), 'W3': array([[4.0291898, 2.6000312]]), 'b3': array([[6.42519352]]), 'W4': array([[305.38023131]]), 'b4': array([[310.63465106]])}


array([[0., 0.],
       [0., 0.],
       [0., 0.],
       ...,
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [154]:
print(parametres)

cible = np.array(['25','60','160','0','1','1800'])

cible = cible.T

predict(cible,parametres)


{'W1': array([[ 1.62434536, -0.61175641, -0.52817175, -1.07296862,  0.86540763],
       [-2.3015387 ,  1.74481176, -0.7612069 ,  0.3190391 , -0.24937038],
       [ 1.46210794, -2.06014071, -0.3224172 , -0.38405435,  1.13376944],
       [-1.09989127, -0.17242821, -0.87785842,  0.04221375,  0.58281521],
       [-1.10061918,  1.14472371,  0.90159072,  0.50249434,  0.90085595]]), 'b1': array([[-0.68372786],
       [-0.12289023],
       [-0.93576943],
       [-0.26788808],
       [ 0.53035547]]), 'W2': array([[-0.69166075, -0.39675353, -0.6871727 , -0.84520564, -0.67124613],
       [-0.0126646 , -1.11731035,  0.2344157 ,  1.65980218,  0.74204416],
       [-0.19183555, -0.88762896, -0.74715829,  1.6924546 ,  0.05080775]]), 'b2': array([[-0.63699565],
       [ 0.19091548],
       [ 2.10025514]]), 'W3': array([[0.12015895, 0.61720311, 0.30017032]]), 'b3': array([[-0.35224985]])}


ValueError: shapes (5,5) and (6,) not aligned: 5 (dim 1) != 6 (dim 0)